# CanvasData - Generate Engagement Data
***This notebook [currently] assumes you have a CanvasData Spark DB as created by the syncCanvasData pipeline.***

*Authors: Departnemt of Education, Tasmania.*


Sample notebook that generates a dataset describing course engagement by user and day of year.

You can complement this with another dataset (e.g. student and school enrolment data, or network information to identify at-school vs at-home requests).

Engagement in this instance is looking at a number of measures including:
- Number of requests (web interactions) performed by the user against the course. Note these can happen in the background if someone e.g. leaves Canvas open.
- Number of Conversations / Messages against the course.
- Number of Quiz Responses
- Number of Assignment(s)
- Boolean flag "Participated" that is True if any of the above conditions are greater than zero.

> **Note**: This is not intended as a complete list of engagement measures. You may wish to modify this notebook to suit your own needs (and ideally submit the changes on GitHub :-) )

Parameters:
- **search_start_date**: Start date for request/engagement data. Data prior to this date will be ignored. Used to manage size of the generated dataset.
- **storage_account**: Short name of your storage account where data should be located.
- **container_name**: Name of the container to store the data in.
- **container_path**: Path to store the new dataset in.
- **create_spark_table**: Boolean to indicate whether a Spark table should be created.
- **refresh_spark_table**: Whether to refresh spark metadata or not. Useful if the schema has changed (e.g. you've added a column)
- **spark_db**: Name of the Spark database.
- **spark_table_name**: Name of the Spark table.

## Notebook Parameters

Change these to suite your environment.

In [ ]:
search_start_date = "2022-01-01" # Start date for request / other data. Used to reduce size of the dataset.

storage_account = '' # Short name of your storage account FQDN (e.g. everything before the dot)
container_name = "stage3np" # Container to store the data. Typically this won't change
container_path = "VleCourseActivity" # Path in container to store the data against. 

create_spark_table = False # Set to TRUE to create a spark table representing the dataset.
refresh_spark_table = True # Set to TRUE to refresh table metadata on completion.
spark_db = "VleData" # Name of your Spark DB where the table should reside. Will be created if it doesn't exist.
spark_table_name = "CanvasData_UserCourseEngagement" # Name of the table you want to create.

## Prepare SparkSQL

> No changes should be required past this point

Here we create several temporary views (session-scoped) to improve readability of the main query further below.

CTE's are an alternative to this but are only supported in later versions of SparkSQL (v3 and up).

In [ ]:
%%pyspark

# This is just used to surface PySpark parameters in SparkSQL
sqlContext.sql(f"SET search_start_date = '{search_start_date}'")

In [ ]:
%%sql
-- Workaround for some schema mapping issues in SparkSQL when reading Parquet.
SET spark.sql.legacy.parquet.int96RebaseModeInRead=CORRECTED

In [ ]:
%%sql
-- Course information
CREATE OR REPLACE TEMPORARY VIEW vwCourses AS (
    SELECT DISTINCT id FROM CanvasData.course_dim cd
    WHERE workflow_state = "available"
    AND   created_at >= ${search_start_date}
);

-- Aggregation of request data
-- Note this may need additional filters to exclude e.g. mobile app based traffic etc. which typically presents as an API call.
-- We may also want to break this down by request type (e.g. GET vs PUT/POST [suggesting interaction vs a submission])
CREATE OR REPLACE TEMPORARY VIEW vwRequestAgg AS (
    SELECT req.user_id, req.course_id, req.timestamp_day, COUNT(course_id) AS request_count
    FROM (SELECT * FROM CanvasData.requests WHERE course_id IN (SELECT id FROM vwCourses) AND timestamp_day >= ${search_start_date}) req
    WHERE course_id IS NOT NULL
    GROUP BY req.user_id, req.course_id, req.timestamp_day
);

-- Messages associated with a course.
CREATE OR REPLACE TEMPORARY VIEW vwConvoMessage AS (
    SELECT p.user_id, p.course_id, date_format(msg.created_at, "yyyy-MM-dd") AS created_day, COUNT(*) AS message_count
    FROM (SELECT * FROM CanvasData.conversation_message_participant_fact WHERE course_id IN (SELECT id FROM vwCourses)) p
    INNER JOIN CanvasData.conversation_message_dim msg ON p.conversation_message_id = msg.id
    GROUP BY user_id, course_id, date_format(msg.created_at, "yyyy-MM-dd")
);

-- Quiz submissions
CREATE OR REPLACE TEMPORARY VIEW vwQuizSubmission AS (
    SELECT user_id, course_id, date_format(date, "yyyy-MM-dd") AS date, SUM(total_attempts) AS quiz_submission_count
    FROM CanvasData.quiz_submission_historical_fact
    WHERE course_id IN (SELECT id FROM vwCourses)
    GROUP BY user_id, course_id, date_format(date, "yyyy-MM-dd")
);

-- General assignment submissions
CREATE OR REPLACE TEMPORARY VIEW vwSub AS (
    SELECT sf.user_id, sf.course_id, date_format(sd.submitted_at, "yyyy-MM-dd") AS submitted_at_day, COUNT(*) AS submission_count
    FROM CanvasData.submission_fact sf
    INNER JOIN CanvasData.submission_dim sd ON sf.submission_id = sd.id
    WHERE course_id IN (SELECT id FROM vwCourses)
    GROUP BY sf.user_id, sf.course_id, date_format(sd.submitted_at, "yyyy-MM-dd")
);

-- List of days in the period
CREATE OR REPLACE TEMPORARY VIEW vwDayList AS (
    SELECT explode(sequence(search_start_date, current_date(), interval 1 day)) as date
);

## Dataset Generation

Finally - generate the dataset and write into nominated location.

Note this is done with PySpark (and not SparkSQL) so we can dynamically set location.

In [ ]:
%%sql
CREATE OR REPLACE TEMPORARY VIEW vwDataSet AS (
    SELECT 
        pseu.unique_name AS user_name,
        u.Name AS user_display_name,
        c.Name AS course_name,
        req.course_id, 
        dl.date, 
        req.request_count AS web_request_count,
        COALESCE(msg.message_count, 0) AS message_count,
        COALESCE(qz.quiz_submission_count, 0) AS quiz_submission_count,
        COALESCE(sub.submission_count, 0) AS submission_count,
        CASE WHEN req.request_count > 0 OR COALESCE(msg.message_count, 0) > 0 OR COALESCE(qz.quiz_submission_count, 0) > 0 OR COALESCE(sub.submission_count, 0) > 0
            THEN TRUE 
            ELSE FALSE 
        END AS participated
    FROM CanvasData.user_dim u
    INNER JOIN CanvasData.pseudonym_dim pseu ON u.id = pseu.user_id AND pseu.workflow_state = 'active' AND pseu.position = 1
    CROSS JOIN vwDayList dl
    LEFT OUTER JOIN vwRequestAgg req ON u.id = req.user_id AND req.timestamp_day = dl.date
    LEFT OUTER JOIN CanvasData.course_dim c ON req.course_id = c.id
    LEFT OUTER JOIN vwConvoMessage msg ON u.id = msg.user_id AND req.course_id = msg.course_id AND req.timestamp_day = msg.created_day
    LEFT OUTER JOIN vwQuizSubmission qz ON u.id = qz.user_id AND req.course_id = qz.course_id AND req.timestamp_day = qz.date
    LEFT OUTER JOIN vwSub sub ON u.id = sub.user_id AND req.course_id = sub.course_id AND req.timestamp_day = sub.submitted_at_day
)

In [ ]:
%%pyspark

dest_path = f"abfss://{container_name}@{storage_account}.dfs.core.windows.net/{container_path}/"

df = spark.sql("SELECT * FROM vwDataSet")

df.write.mode('overwrite').parquet(dest_path)

In [ ]:
%%pyspark

table_name = f"{spark_db}.{spark_table_name}"

# Create our Spark Table if required.
if create_spark_table:
    spark.sql(f"CREATE DATABASE IF NOT EXISTS {spark_db}")
    spark.sql(f"CREATE TABLE IF NOT EXISTS {table_name} USING PARQUET LOCATION '{dest_path}'")

if refresh_spark_table:
    spark.sql(f"REFRESH TABLE {table_name}")